# 03 fastTextのword embeddingsを使ってみる
注. このnotebookは、Google Colabではなく、手元の環境で動かすことを想定しています。
　　（Google Colabで動かすとかなり時間がかかると思います。）

* fasttextの単語ベクトルを使って、文書のベクトル表現を得る方法を、以下に示します。
 * 各文書は、それに含まれる単語の単語ベクトルの平均として、ベクトル化されます。

## 03-00 準備
* 必要ならば、このnotebookを実行する前に、condaの環境を作っておきましょう。

```sh
$ conda create -n wordvec
$ conda activate wordvec
$ conda install ipykernel
$ python -m ipykernel install --user --name wordvec --display-name wordvec
```

* そして、必要なパッケージは前もってインストールしておきましょう。

```bash
$ pip install fasttext
$ pip install ml-datasets
```

## 03-01 fastTextの単語ベクトルを取得する

### "Word vectors for 157 languages"から英語データをダウンロード
* fastTextのドキュメント https://fasttext.cc/docs/en/crawl-vectors.html
* 論文 https://arxiv.org/abs/1802.06893

In [3]:
import fasttext.util

fasttext.util.download_model('en', if_exists='ignore')

'cc.en.300.bin'

## 03-02 IMDbデータセットをダウンロード

### 本家のサイトからダウンロード
* 方法は他にもあるが、ここでは本家サイトから直にダウンロードする。

In [4]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2020-11-26 22:13:47--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  11.2MB/s    in 17s     

2020-11-26 22:14:04 (4.70 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [5]:
!tar zxf aclImdb_v1.tar.gz

### fastTextの単語ベクトルを読み込む
* さきほどダウンロードし、解凍しておいたものを読み込む。

In [10]:
import fasttext

model_path = 'cc.en.300.bin'
print(f'# loading {model_path} ...', flush=True) 
ft = fasttext.load_model(model_path)

# loading cc.en.300.bin ...


### IMDbデータセットを読み込む
* 本家サイトからダウンロードし、解凍しておいたものを、ml_datasetsを使って読み込む。

In [11]:
from ml_datasets import imdb

train_valid_data, test_data = imdb(loc='./aclImdb')

In [12]:
train_valid_data[:3]

[("This long episode packs amount of astounding of surprises, thriller, mystery and concerns about a battle of wits of Sherlock against Charles Augustus Milverton, a master blackmailer. This is an excellent overlong runtime of Jeremy Brett-Holmes series. In the film appear usual Holmes's cannon as Inspector Lestrade and Mrs Hudson, though no Moriarty, however is a greatest villain, Charles Augustus.\n\n\n\nIt's a genuine ripping yarn with intrigue, thrills, and suspense, including an exciting final twist. This is a particular Sherlock movie but we find to Holmes falling in love with a servant, kissing, crying and even robbing. This time along with the episode ¨Scandal in Boheme¨ with Irene Adler, result to be the only one which Holmes is enamored. Top-notch Brett performance, he alongside Peter Cushing are the best Sherlock TV , while in the cinema is forever Basil Rathbone. Brett performs as a resolutive, headstrong, impetuous sleuth. Here Doctor Watson isn't a comic, botcher, and clu

### テキスト部分と0/1ラベル部分に分ける

In [13]:
train_valid_texts, train_valid_labels = zip(*train_valid_data)
test_texts, test_labels = zip(*test_data)

### テストセット以外をランダムにシャッフル

In [14]:
import random

random.seed(123)
random.shuffle(train_valid_data)

### 手動で訓練データと検証データへ分割

In [15]:
split = int(len(train_valid_data) * 0.8)
train_texts, train_labels = train_valid_texts[:split], train_valid_labels[:split]
valid_texts, valid_labels = train_valid_texts[split:], train_valid_labels[split:]

In [16]:
print(f'# {len(train_texts)} training, {len(valid_texts)} validation, and {len(test_texts)} test docs')

# 20000 training, 5000 validation, and 25000 test docs


In [17]:
splits = {
    'train': (train_texts, train_labels),
    'valid': (valid_texts, valid_labels),
    'test': (test_texts, test_labels)
}

### 全文書のembeddingを得てファイルに保存
* fastTextのget_sentence_vectorを使って文書のベクトル表現を得る。
* 全文書のベクトル表現をndarrayに変換、`.npy`形式で保存
* 全文書のラベルもndarrayに変換、`.npy`形式で保存

In [18]:
import numpy as np

for tag in splits:
    print(f'# {tag} set: ', end='', flush=True)
    cnt = 0
    X = list()
    for text in splits[tag][0]:
        vec = ft.get_sentence_vector(' '.join(text.split('\n')))
        X.append(vec)
        cnt += 1
        if cnt % 10000 == 0: print('*', end='', flush=True)
        elif cnt % 1000 == 0: print('-', end='', flush=True)
    X = np.array(X)
    with open(f'{tag}.npy', 'wb') as f:
        np.save(f, X, allow_pickle=False)
    with open(f'{tag}_labels.npy', 'wb') as f:
        np.save(f, np.array(splits[tag][1]), allow_pickle=False)
    print(flush=True)

# train set: ---------*---------*
# valid set: -----
# test set: ---------*---------*-----


In [19]:
!ls -al *.npy

-rw-rw-rw- 1 masada Domain Users   200128 Nov 26 22:16 test_labels.npy
-rw-rw-rw- 1 masada Domain Users 30000128 Nov 26 22:16 test.npy
-rw-rw-rw- 1 masada Domain Users   160128 Nov 26 22:16 train_labels.npy
-rw-rw-rw- 1 masada Domain Users 24000128 Nov 26 22:16 train.npy
-rw-rw-rw- 1 masada Domain Users    40128 Nov 26 22:16 valid_labels.npy
-rw-rw-rw- 1 masada Domain Users  6000128 Nov 26 22:16 valid.npy
